In [ ]:
from lib.utils import *
from lib.models import *
from lib.ekyn import *
from lib.env import *
from lib.datasets import *

In [ ]:
testloader = DataLoader(Dataset2p0(dir='w9_ss/test',labels='w9_ss/y_test.pt'),batch_size=32,shuffle=True)

print(f'testloader: {len(testloader)} batches')

In [ ]:
criterion = torch.nn.CrossEntropyLoss()

class Gandalf(nn.Module):
    def __init__(self) -> None:
        super().__init__()
        self.encoder = Frodo(n_features=5000,device=DEVICE).to(DEVICE)
        self.lstm = nn.LSTM(16,32)
        self.fc1 = nn.Linear(32,3)
    def forward(self,x_2d,classification=True):
        x_2d = x_2d.view(-1,9,1,5000)
        x = torch.Tensor().to(DEVICE)
        for t in range(x_2d.size(1)):
            xi = self.encoder(x_2d[:,t,:,:],classification=False)
            x = torch.cat([x,xi.unsqueeze(0)],dim=0)
        out,_ = self.lstm(x)
        if(classification):
            x = self.fc1(out[-1])
        else:
            x = out[-1]
        return x
model = Gandalf()
model.load_state_dict(torch.load(f=f'../lstm_w9_ss/best_model.pt'))
model.to(DEVICE)

In [ ]:
loss,metrics,y_true,y_pred,_ = evaluate(testloader,model,criterion,DEVICE)

In [ ]:

cm_grid(y_true=y_true,y_pred=y_pred,save_path='cm.pgf')
print(metrics)
print(loss)

In [ ]:
with torch.no_grad():
    y_true = torch.Tensor()
    y_pred = torch.Tensor()
    y_logits = torch.Tensor()
    X_all = torch.Tensor()
    loss_total = 0
    # for (Xi,yi) in tqdm(testloader):
    for i in range(200):
        Xi,yi = next(iter(testloader))
        y_true = torch.cat([y_true,yi.argmax(axis=1)])

        Xi,yi = Xi.to(DEVICE),yi.to(DEVICE)
        logits = model(Xi,classification=False)
        # loss = criterion(logits,yi)
        # loss_total += loss.item()
        
        # y_logits = torch.cat([y_logits,torch.softmax(logits,dim=1).detach().cpu()])
        y_logits = torch.cat([y_logits,logits.detach().cpu()])
        X_all = torch.cat([X_all,Xi.detach().cpu()])
        # y_pred = torch.cat([y_pred,torch.softmax(logits,dim=1).argmax(axis=1).detach().cpu()])


In [ ]:
from sklearn.manifold import TSNE
tsne = TSNE(n_components=2,init='pca')
logits_tsne = tsne.fit_transform(y_logits.detach().cpu())

In [ ]:
df = pd.DataFrame(logits_tsne)
df['y'] = y_true
sns.scatterplot(data=df,x=0,y=1,hue='y')
plt.savefig('tsne.jpg')

In [ ]:
tsne = TSNE(n_components=2,init='pca')
logits_tsne = tsne.fit_transform(X_all.detach().cpu())

In [ ]:
df = pd.DataFrame(logits_tsne)
df['y'] = y_true
sns.scatterplot(data=df,x=0,y=1,hue='y')
plt.savefig('tsne_raw.jpg')